# Training of LSTM 

This Notebook is embedding the sequences of the Patients. These embedded sequences are then used for the Training of various LSTM models. 

In [ ]:
# lstm autoencoder to recreate a timeseries
#gridsearch: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
#https://machinelearningmastery.com/how-to-grid-search-deep-learning-models-for-time-series-forecasting/
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from tensorflow import keras
import pickle
import pyarrow.parquet as pq
from gensim.models import Word2Vec
import umap
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns
import pandas as pd
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn import preprocessing
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####

import time
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from functools import reduce
from ppca import PPCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
import json
from sklearn import metrics
from sklearn.decomposition import FastICA
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from pickle import load
from pickle import dump
import pickle
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing
import scipy.cluster.hierarchy as shc
import scipy.stats as stats
import researchpy as rp
from keras.models import Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split  
from keras.layers import Input, Dense 
from keras.models import Model, Sequential 
from keras import regularizers 
import umap
from sklearn.cluster import DBSCAN
import hdbscan
from statistics import mean 

# Load embedded sequence 

In [ ]:
#Timeseries_per_patient_LSTM_Data_embedded_cbow_dim20_win5_mc0.txt
def load_dataset(per_day,embedding_method,embedding_size):
    if per_day: 
        df_name='Timeseries_per_patient_per_day_LSTM_Data_embedded_'
    else: 
        df_name='Timeseries_per_patient_LSTM_Data_embedded_'
    if embedding_method=='cbow':
        df_name=df_name+'cbow_dim{}_win5_mc0'.format(embedding_size)
    if embedding_method=='skipgram':
        df_name=df_name+'skipgram_dim{}_win5_mc0'.format(embedding_size)
    print(df_name)
    with open("Cohort/Time_Series/"+df_name+'.txt', "rb") as fp:   # Unpickling
        data = pickle.load(fp)
    data_sample= data[:300]
    return data,data_sample, df_name

# Configuration of the Dataset

In [ ]:
#configure dataset that should be used 
per_day=False
embedding_method='cbow'
embedding_size='20'

data,sample,df_name=load_dataset(per_day,embedding_method,embedding_size)


In [ ]:
data_structure='per_patient'#'per_patient_per_day'
timesteps=len(data[0])
n_features=len(data[0][0])
layer_size_1=32
layer_size_2=16
activation_func='tanh'
optimizer_func='adam'
loss_func='mse'
n_epochs=3
n_batch_size=100
X=sample

# define the model 

In [ ]:

model = Sequential()
model.add(LSTM(layer_size_1, activation=activation_func, input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(layer_size_2, activation=activation_func, return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(layer_size_2, activation=activation_func, return_sequences=True))
model.add(LSTM(layer_size_1, activation=activation_func, return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer=optimizer_func, loss=loss_func,metrics=[loss_func])
model.summary()

# Fit Model 

In [ ]:
# fit model


hist=model.fit(X, X, epochs=n_epochs, batch_size=n_batch_size, verbose=1)
# demonstrate reconstruction
#print(history.history['val_loss'][(epochs-1)])
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))


In [ ]:
loss_value=hist.history[loss_func][n_epochs-1]

In [ ]:
result=pq.read_table('Cohort/Metrics_LSTM.parquet').to_pandas()
result=result.append({'data_structure':data_structure,'timesteps':timesteps,'n_features':n_features,'layer_size_1':layer_size_1,'layer_size_2':layer_size_2,'activation_func':activation_func,'optimizer_func':optimizer_func,'loss_func':loss_func,'n_epochs':n_epochs,'n_batch_size':n_batch_size,'loss':loss_value}, ignore_index=True)
result.to_parquet('Cohort/Metrics_LSTM.parquet')
result

In [ ]:
result=pq.read_table('Cohort/Metrics_LSTM.parquet').to_pandas()
result

In [ ]:
experiment_name=df_name+'_'+data_structure+str(layer_size_1)+'_'+str(layer_size_2)+'_'+activation_func+'_'+optimizer_func+'_'+loss_func+'_'+str(n_epochs)+'_'+str(n_batch_size)

In [ ]:
model.save('Cohort/LSTM_Models/'+experiment_name)

# First Test for  prediction 

In [ ]:
model = keras.models.load_model('Cohort/LSTM_Models/'+experiment_name)

In [ ]:
hidden_representation = Sequential() 
hidden_representation.add(model.layers[0]) 
hidden_representation.add(model.layers[1])  
#hidden_representation.add(model.layers[2])
normal_hidden_rep = hidden_representation.predict(X)

In [ ]:
normal_hidden_rep

In [ ]:
len(normal_hidden_rep)

In [ ]:
len(normal_hidden_rep[0])

# Clustering 

In [ ]:
# get mrns  can
timeseries_per_patient = pq.read_table('Cohort/Time_Series/time_series_per_patient_mrns.parquet').to_pandas()
#timeseries_per_patient=timeseries_per_patient.drop('unique_concept', axis=1)
#timeseries_per_patient
timeseries_per_patient_sample=timeseries_per_patient.head(300)
#timeseries_per_patient_sample

In [ ]:
mrn=timeseries_per_patient['medical_record_number'].to_list()
mrn

In [ ]:
def apply_umap(transformed_df,dimension):
    clusterable_embedding = umap.UMAP(
        n_neighbors=50,
        min_dist=0.1,
        n_components=dimension,
        random_state=42,
    )
    X_transformed=clusterable_embedding.fit_transform(transformed_df)
    return X_transformed

In [ ]:
def scatter_plot(df,labels):
    sns.set(style='white', rc={'figure.figsize':(10,8)})
    plt.scatter(df[:, 0], df[:, 1], c=labels, s=0.1, cmap='Spectral');
    plt.show()

In [ ]:
def apply_kmeans(transformed_sample,ellbow_method,cluster):
    if ellbow_method==True:
        elbow_method(transformed_sample)
    #scatter_plot(transformed_sample,None) 
    #plt.scatter(transformed_sample[:,0],transformed_sample[:,1])
    kmeans = KMeans(n_clusters=cluster, init='k-means++', max_iter=5000, n_init=10, random_state=0)
    pred_y = kmeans.fit_predict(transformed_sample)
    #plt.scatter(transformed_sample[:,0], transformed_sample[:,1])
    #plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red')
    #plt.show()
    #scatter_plot(transformed_sample,kmeans.labels_)
    '''
    from sklearn.metrics.pairwise import pairwise_distances_argmin
    fig = plt.figure(figsize=(15, 5))
    fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
    colors = ['#4EACC5', '#FF9C34', '#4E9A06','#FF0000','#8800FF']
    k_means_labels = pairwise_distances_argmin(transformed_sample, kmeans.cluster_centers_)
    ax = fig.add_subplot(1, 3, 1)
    for k, col in zip(range(cluster), colors):
        my_members = k_means_labels == k
        cluster_center = kmeans.cluster_centers_[k]
        ax.plot(transformed_sample[my_members, 0], transformed_sample[my_members, 1], 'w',
                markerfacecolor=col, marker='.')
        ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
                markeredgecolor='k', markersize=6)
    experiment_name=experiment_name
    ax.set_title(experiment_name)
    ax.set_xticks(())
    ax.set_yticks(())
    fig.savefig('Cohort/Models/Plots/'+experiment_name+'.png')'''
    return kmeans.labels_

In [ ]:
def elbow_method(transformed_sample): 
    wcss = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        kmeans.fit(transformed_sample)
        wcss.append(kmeans.inertia_)
    plt.plot(range(1, 11), wcss)
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()

In [ ]:
def get_silhouette_Coefficient(labels,df):
    m=metrics.silhouette_score(df, labels, metric='euclidean')
    print('silhouette_score:',m)
    return m

In [ ]:
n_cluster= 4 
#prepare data for plotting 
df_dim_red_plot=apply_umap(normal_hidden_rep,2)        
#print first 2 dim of dimensionality reduced data:
scatter_plot(df_dim_red_plot,None)
labels=apply_kmeans(normal_hidden_rep,True,n_cluster)
scatter_plot(df_dim_red_plot,labels)

In [ ]:
get_silhouette_Coefficient(labels,normal_hidden_rep)

In [ ]:
labels

# Feature preprocessing

In [ ]:
#function for statistics: 
def get_base_characteristic_value(df , characteristic , kind):    
    if kind=="mean": 
        df_mean=df[characteristic].mean()
        df_std= df[characteristic].std()
        df_max= df[characteristic].max()
        df_min= df[characteristic].min()
        base_characteristics_cohort=pd.DataFrame({'Variable': [characteristic+"_mean", characteristic+"_std", characteristic+"_max", characteristic+"_min"],
                                                  'Value': [(df_mean/365), (df_std/365), (df_max/365), (df_min/365)],})
       
    if kind=="count":
        base_characteristics_cohort=pd.DataFrame(columns=["Variable","Value"])
        feature_value=df[characteristic].unique()
        #print(feature_value)
        for value in feature_value: 
            df_condition=df.loc[df[characteristic]==value]
            df_percent= df_condition.shape[0]/df.shape[0]
            #print(df_percent)
            new_row1 = {'Variable': value+"_total",'Value': df_condition.shape[0]}
            new_row2 = {'Variable': value+"_relation",'Value': df_percent}
            base_characteristics_cohort=base_characteristics_cohort.append(new_row1, ignore_index=True)
            base_characteristics_cohort=base_characteristics_cohort.append(new_row2, ignore_index=True)
       # print(df_condition.shape[0], df_percent)
    #print (base_characteristics_cohort)
    return base_characteristics_cohort

def get_base_characteristics(df, characteristics): 
    base_characteristics_cohort=pd.DataFrame(columns=["Variable","Value"])
    for characteristic in characteristics:
        intermediate_base_characteristics_cohort=get_base_characteristic_value(df,characteristic[0],characteristic[1])
        base_characteristics_cohort=pd.concat([base_characteristics_cohort,intermediate_base_characteristics_cohort])
    print(base_characteristics_cohort)
    return base_characteristics_cohort

def get_cluster_information(df):
    dim_red_method=df
    dim_red_method='LSTM_labels'
    base_characteristics=[
        [ "HF_Onset_age_in_days","mean"],
        ["gender","count"]
        ]
    baseline_characteristics=[]
    for cluster in df[dim_red_method].unique(): 
        cluster_characteristics=[]
        df_temp=df.loc[df[dim_red_method] == cluster]
        df_base_characteristics=get_base_characteristics(df_temp, base_characteristics)
        
        cluster_characteristics.append(cluster)
        cluster_characteristics.append(len(df_temp))
        cluster_characteristics.append(df_base_characteristics)
        baseline_characteristics.append(cluster_characteristics)
    return baseline_characteristics
        #print(str(cluster))
        #print(len(df_temp))
        
        #print(df_temp_baseline)

In [ ]:
def num_feature_importance_anova(df,ctransformer,dim_red_method,n_cluster,top_features):
    df_temp=df
    #replace cluster names 
    for cluster in (range(n_cluster)):
        cluster_name='cluster_'+str(cluster)
        df[dim_red_method].replace({cluster: cluster_name},inplace=True)
    #normalize num columns
    min_max_scaler = preprocessing.MinMaxScaler()
    num_columns=ctransformer.transformers[0][2]
    df_temp[list(num_columns)] = min_max_scaler.fit_transform(df_temp[list(num_columns)])
    #iterate over num columns and calculate the p-Value: 
    col=['column name','F-Value','p-value','absolute_p','compared to other']
    result_all=pd.DataFrame(columns=col) 
    result_anova=[]
    for cluster in df_temp[dim_red_method].unique():
        result_all=pd.DataFrame(columns=col)
        df_temp['temp_cluster']=df_temp[dim_red_method]
        df_temp.loc[df[dim_red_method] != cluster, "temp_cluster"] = "other_cluster"
        for num_col in num_columns: 
            feature=num_col
            result = df_temp.groupby('temp_cluster')[feature].apply(list)
            #print(result)
            feature_value_1=result[cluster]
            #print(feature_value_1)
            feature_value_2=result['other_cluster']
            mean_1=mean(feature_value_1)
            mean_2=mean(feature_value_2)
            if mean_1 > mean_2: 
                compared='higher'
            else:
                compared='lower'
            #print(len(result['cluster_3']))
            #print(len(result['cluster_0']))
            F, p = stats.f_oneway(*result)
            p=format(p, '.300000000g')
            p=float(p)
            if p!=0:
                importance=abs(np.log(p))
            else: 
                importance=0
            row={'column name':(feature+'_'+cluster),'F-Value':F,'p-value':p,'absolute_p':importance,'compared to other':compared}
            result_all=result_all.append(row, ignore_index=True)
        result_all=result_all.sort_values(by=['absolute_p'],ascending=False)
        result_anova.append(result_all)
    #result_all=result_all.drop_duplicates(subset='column name',keep='first', inplace=False)
    #return result_all.head(top_features)
    return result_anova

In [ ]:
def cat_feature_importance(df,ctransformer,sup_colums,dim_red_method,n_cluster,top_features):
    #replace cluster names 
    #establish two categories in all Categories 
    
   
    for cluster in (range(n_cluster)):
        cluster_name='cluster_'+str(cluster)
        df[dim_red_method].replace({cluster: cluster_name},inplace=True)
    df=df.replace(True, 'Yes')
    df=df.replace(False,'No')
    df=df.fillna('No')
    df=df.replace(1, 'Yes')
    df=df.replace(0,'No')
    df=df.fillna('No')
    col=['column name','Pearson Chi-square','Cramers V','p-value','absolute_p','compared to other']
    result_all=pd.DataFrame(columns=col)
    result_chi=[]
    for cluster in df[dim_red_method].unique():
        result_all=pd.DataFrame(columns=col)
        df['temp_cluster']=df[dim_red_method]
        df.loc[df[dim_red_method] != cluster, "temp_cluster"] = "other_cluster"
        #print(df[[dim_red_method,'temp_cluster']])     
        cat_columns=ctransformer.transformers[1][2]
        #iterate over cat columns and calculate the p-Value: 
        for cat_col in cat_columns: 
            feature=cat_col
            crosstab, test_results, expected = rp.crosstab(df[feature], df['temp_cluster'],
                                                   test= "chi-square",
                                                   expected_freqs= True,
                                                   prop= "cell")
            p=format(test_results["results"][1], '.300000000g')
            #print(p)
           # if test_results["results"][1]!=0:
            p=float(p)
            if p!=0:

                importance=abs(np.log(p))
            else: 
                importance=0
            compared=''
            if feature !='gender':
                feature_count_1=len(df.loc[df['temp_cluster']==cluster])
                feature_cluster=df.loc[df['temp_cluster']==cluster]
                feature_percentage_1=(len(feature_cluster.loc[feature_cluster[feature]=='Yes'])/feature_count_1)
                #print(feature_percentage_1)
    
                feature_count_2=len(df.loc[df['temp_cluster']=='other_cluster'])
                feature_cluster_2=df.loc[df['temp_cluster']=='other_cluster']
                feature_percentage_2=(len(feature_cluster_2.loc[feature_cluster_2[feature]=='Yes'])/feature_count_2)
                #print(feature_percentage_2)
                if feature_percentage_1 > feature_percentage_2: 
                    compared='higher'
                else:
                    compared='lower'
            row={'column name':(feature+'_'+cluster),'Pearson Chi-square':test_results["results"][0],'Cramers V':test_results["results"][2],'p-value':p,'absolute_p':importance,'compared to other':compared}
            #row={'column name':feature,'Pearson Chi-square':test_results["results"][0],'Cramers V':test_results["results"][2],'p-value':p,'absolute_p':importance}
            result_all=result_all.append(row, ignore_index=True)
        for cat_col in sup_colums: 
            feature=cat_col
            crosstab, test_results, expected = rp.crosstab(df[feature], df['temp_cluster'],
                                                   test= "chi-square",
                                                   expected_freqs= True,
                                                   prop= "cell")
            #print(crosstab)
            p=format(test_results["results"][1], '.300000000g')
            #print(p)
           # if test_results["results"][1]!=0:
            p=float(p)
            if p!=0:

                importance=abs(np.log(p))
            else: 
                importance=0
            compare=''
            if feature !='gender':
                feature_count_1=len(df.loc[df['temp_cluster']==cluster])
                feature_cluster=df.loc[df['temp_cluster']==cluster]
                feature_percentage_1=(len(feature_cluster.loc[feature_cluster[feature]=='Yes'])/feature_count_1)
               # print(feature_percentage_1)
    
                feature_count_2=len(df.loc[df['temp_cluster']=='other_cluster'])
                feature_cluster_2=df.loc[df['temp_cluster']=='other_cluster']
                feature_percentage_2=(len(feature_cluster_2.loc[feature_cluster_2[feature]=='Yes'])/feature_count_2)
               # print(feature_percentage_2)
                if feature_percentage_1 > feature_percentage_2: 
                    compared='higher'
                else:
                    compared='lower'
            row={'column name':(feature+'_'+cluster),'Pearson Chi-square':test_results["results"][0],'Cramers V':test_results["results"][2],'p-value':p,'absolute_p':importance,'compared to other':compared}
            #row={'column name':feature,'Pearson Chi-square':test_results["results"][0],'Cramers V':test_results["results"][2],'p-value':p,'absolute_p':importance}
            result_all=result_all.append(row, ignore_index=True)
        result_all=result_all.sort_values(by=['absolute_p'],ascending=False)
        result_chi.append(result_all)
    #result_all=result_all.drop_duplicates(subset='column name',keep='first', inplace=False)
    #return result_all.head(top_features)
    return result_chi

In [ ]:
def plotTopFeatures(df,merge_w_supervised,dim_red_method, evaluation_results,  n_cluster, n_topFeatures): 
    #convert the dataframe
    df_origin=df
   # df_origin=pq.read_table(df_path).to_pandas()
    #print(df_origin['gender'])
    df_origin[dim_red_method]=df[dim_red_method]
    conv_df=df_origin
    if merge_w_supervised==True:
        df_supervised_merge= pq.read_table('Cohort/Feature_Extraction/ALL_HF_cohort_supervised_only_ever_diag_drugFORMerge.parquet').to_pandas()
        conv_df.index = conv_df.index.map(str)
        df_supervised_merge.index = df_supervised_merge.index.map(str)
        conv_df=pd.merge(conv_df, df_supervised_merge, left_on='medical_record_number', right_on='medical_record_number')
        
    conv_df=conv_df.replace(True, 1)
    conv_df=conv_df.replace(False,0)
    conv_df=conv_df.replace('yes', 1)
    conv_df=conv_df.replace('no',0)
    conv_df=conv_df.fillna(0)
    conv_df=conv_df.sort_values(by=[dim_red_method],ascending=True)
    #get top featrues: 
    evaluation_pandas=evaluation_results
    features=getTopCluster(evaluation_pandas, n_topFeatures, n_cluster)
    #plot features 
    #print (cluster_name)
    plot_risk_groups(conv_df, features, dim_red_method,friendly_names_converter=None, filename='', nrows=1, figsize=[24,10])
    return features
#https://github.com/hpi-dhc/robotehr/blob/e3673aef701aa817c74d04170986f01fa191212a/robotehr/evaluation/risk_groups.py#L70-L100
def plot_risk_groups(df, features,dim_red_method, friendly_names_converter=None, filename='', nrows=1, figsize=[12,3]):
    ncols = int(len(features) / nrows)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    fig.tight_layout(pad=3.0)

    for i in range(len(features)):
        row_index = int(i / ncols)
        col_index = i % int(len(features) / nrows)

        current_axis = ax[row_index][col_index] if nrows > 1 else ax[col_index]
        if df[features[i]].min() == 0 and df[features[i]].max() == 1:
            current_axis.set_ylim(bottom=-0.5, top=1.5)
        sns.violinplot(
            x=dim_red_method,
            y=features[i],
            data=df,
            palette="muted",
            ax=current_axis,
            #hue='gender'
        )
        if friendly_names_converter:
            title = friendly_names_converter.get(features[i])
        else:
            title = features[i]
        if len(title) > 50:
            title = f'{title[:50]} ...'
        current_axis.set_title(f'{title}', fontsize=11)
        current_axis.set_xlabel('')
        current_axis.set_ylabel('')
    if filename:
        fig.savefig(filename, dpi=300, bbox_inches="tight")
    return fig
def getTopCluster(evaluation_pandas, n_topFeature, n_cluster ): 
    topFeatures=[]
    for n in range(n_cluster):
        print(n)
        features=[]
         #categorical features
        features=evaluation_pandas[2][n]['column name'].values
        x=0
        for i in range(n_topFeature):
            feature=subStringCluster(features[x])
            if feature != 'gender' :
                topFeatures.append(subStringCluster(features[x]))
            if 'Procedure' in feature: 
                print (feature)
                x=x+1
                print(subStringCluster(features[x]))
                topFeatures.append(subStringCluster(features[x]))
                
            else: 
                x=x+1
                #print(features)
                topFeatures.append(subStringCluster(features[x]))
            x=x+1
        
        #numeric
        features=evaluation_pandas[1][n]['column name'].values 
        for i in range(n_topFeature):
            topFeatures.append(subStringCluster(features[n]))
    topFeatures=set(topFeatures)
    topFeatures=list(topFeatures)
    print(topFeatures)
    return topFeatures
def subStringCluster(string):
    a_string=string
    split_string=a_string.split('_cluster_',1)
    substring = split_string[0]
    return substring
        
    
    

# Get Cluster insights

In [ ]:
import statistics
# load static feature Cohort: 
df_origin= pq.read_table('Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab.parquet').to_pandas()
df_origin.loc[(df_origin['HF_Onset_age_in_days'] > 32850),'HF_Onset_age_in_days']=32850
#merge original dataframe with supervised features:
df_supervised_merge= pq.read_table('Cohort/Feature_Extraction/ALL_HF_cohort_supervised_only_ever_diag_drugFORMerge_wLab.parquet').to_pandas()
df_origin.index = df_origin.index.map(str)
df_supervised_merge.index = df_supervised_merge.index.map(str)
sup_colums=df_supervised_merge.columns
df_cohort=pd.merge(df_origin, df_supervised_merge, left_on='medical_record_number', right_on='medical_record_number')
#merge with inpatient 
#df_inpatient_merge= pq.read_table('Cohort/Feature_Extraction/Supervised_ALL_HF/inpatient_events_merge.parquet').to_pandas()
df_cohort.index = df_cohort.index.map(str)
#df_inpatient_merge.index = df_inpatient_merge.index.map(str)
#inp_colums=df_inpatient_merge.columns
#df_cohort=pd.merge(df_cohort, df_inpatient_merge, left_on='medical_record_number', right_on='medical_record_number')
#keep Patient with Time Series: 
df_cohort=df_cohort[df_cohort.index.isin(mrn)]
# add cluster to Patients: 
df_cohort['LSTM_labels']=labels
#df_cohort
evaluation_results=[]
#get general cluster information
cluster_information=get_cluster_information(df_cohort)
evaluation_results.append(cluster_information)
#ANOVA 
#load one ctransformer:
with open('Cohort/Models/ColumnTransformer/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned.parquet_MinMaxScaler_BinaryEncoder.pkl', 'rb') as f:
            ctransformer = pickle.load(f)
ctransformer
top_numerical_features_anova=num_feature_importance_anova(df_cohort,ctransformer,'LSTM_labels',n_cluster,5)
print('Top Numerical features: \n',top_numerical_features_anova)
evaluation_results.append(top_numerical_features_anova)
evaluation_results
top_catigorical_features=cat_feature_importance(df_cohort,ctransformer,sup_colums,'LSTM_labels',n_cluster,5)
print('Top Categorical features: \n',top_catigorical_features)
evaluation_results.append(top_catigorical_features)

             

In [ ]:
df_cohort

In [ ]:
plotTopFeatures(df_cohort,True,'LSTM_labels', evaluation_results,n_cluster , 1)

In [ ]:
data_structure='per_patient'#'per_patient_per_day'
timesteps=len(data[0])
n_features=len(data[0][0])
layer_size_1=32
layer_size_2=16
activation_func='sigmoid'
optimizer_func='adam'
loss_func='mse'
n_epochs=30
n_batch_size=10
X=sample

In [ ]:
col=['data_structure','timesteps','n_features','layer_size_1','layer_size_2','activation_func','optimizer_func','loss_func','n_epochs','n_batch_size','loss']
result=pd.DataFrame(columns=col)
result.to_parquet('Cohort/Metrics_LSTM.parquet')

In [ ]:
result=pq.read_table('Cohort/Metrics_LSTM.parquet').to_pandas()
result